# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from Config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
#CSV
chart_data = pd.read_csv('../output_data/weather_city.csv')
vac_df = pd.DataFrame(chart_data)


,city,lat,lon,max temp,humidity,cloudiness,wind_speed,country,date
95,jomalig,13.93,121.42,79.43,78,100,1.74,PH,1596334159
133,darhan,11.00,122.67,83.26,73,100,4.47,PH,1596334344
307,barbacoas,12.68,124.36,82.99,69,100,3.44,PH,1596334388
356,plettenberg bay,14.35,121.02,82.40,83,100,6.93,PH,1596334400
357,mvomero,6.52,124.58,85.89,64,67,2.95,PH,1596334400
375,calama,9.56,122.49,85.44,63,95,2.77,PH,1596334404
454,isperih,9.25,118.41,84.20,74,100,5.26,PH,1596334423
476,zhangye,16.13,119.78,77.72,88,100,8.95,PH,1596334429


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [29]:
# Use the Lat and Lng as locations 
locations = vac_df[["lat", "lon"]]

# Humidity as the weight
humidity = vac_df["humidity"].astype(float)

In [30]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
#locate vacation spots in the Philippians
desired_df = vac_df.loc[(vac_df["max temp"] < 80) & (vac_df["max temp"] > 70) & \
                       (vac_df["wind_speed"] < 10) & (vac_df["cloudiness"] == 0)]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
# set up additional columns to hold information
desired_df["hotel name"] = ""
desired_df.head()

c:\users\simo8\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,lat,lon,max temp,humidity,cloudiness,wind_speed,country,date,hotel name
27,vigrestad,-23.12,-134.97,73.80,78,0,5.19,PF,1596334142,
70,port blair,30.76,20.23,76.35,91,0,8.75,LY,1596333857,
90,albany,-8.93,125.41,70.93,38,0,4.29,TL,1596334158,
101,chuy,36.41,64.91,75.00,28,0,1.34,AF,1596334161,
131,east london,14.90,-24.50,77.45,84,0,3.24,CV,1596334343,


In [41]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "lat,lon", 
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in desired_df.iterrows():

#     # get restaurant type from df
#     restr_type = row['ethnicity']

#     # add keyword to params dict
#     params['keyword'] = restr_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        desired_df.loc[index, 'name'] = results[0]['name']
        desired_df.loc[index, 'address'] = results[0]['vicinity']
        desired_df.loc[index, 'price_level'] = results[0]['price_level']
        desired_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

NameError: name 'restr_type' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
